# Managing underwater positions

In [5]:
import pandas as pd
import vectorbtpro as vbt
from datetime import datetime, timedelta
import pytz
import pandas_ta as ta
import logging
from backtesting import Strategy, Backtest
logging.basicConfig(level=logging.WARNING, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [6]:
symbols = ["SOLUSDT", "BNBUSDT", "ETHUSDT", "BTCUSDT"]
start = "2018-01-01"
# Enter your parameters here
metric = 'total_return'

start_date = datetime(2020, 1, 1, tzinfo=pytz.utc)  # time period for analysis, must be timezone-aware
end_date = datetime.now(pytz.utc)
# end_date = datetime(2020, 1, 1, tzinfo=pytz.utc)

# The following is the number of days to look back for the analysis
time_buffer = timedelta(days=100)  # buffer before to pre-calculate SMA/EMA, best to set to max window
freq = '1h'

vbt.settings.portfolio['init_cash'] = 100_000.  # 100,000$
vbt.settings.portfolio['fees'] = 0.0025  # 0.25%
# vbt.settings.portfolio['slippage'] = 0.0025  # 0.25%

# get binance data doing it this way allows for you to update your data rather than re-downloading it
# binance_data = vbt.BinanceData.fetch(symbols,timeframe=freq, start=start_date,end="now UTC")
# binance_data.save("binance_data.pkl")

# If you already have the data downloaded, you can load it
binance_data = vbt.BinanceData.load("binance_data.pkl")
# binance_data = binance_data.update() if you want to update it.

In [7]:
bnb_data = binance_data.data["BNBUSDT"]
sol_data = binance_data.data["SOLUSDT"]
btc_data = binance_data.data["BTCUSDT"]
eth_data = binance_data.data["ETHUSDT"]
bnb_data = binance_data.data["BNBUSDT"]
sol_data = binance_data.data["SOLUSDT"]
btc_data = binance_data.data["BTCUSDT"]
eth_data = binance_data.data["ETHUSDT"]
# bnb_data.drop(columns=["Close time", "Quote volume", "Number of trades", "Taker base volume", "Taker quote volume"], inplace=True)
# sol_data.drop(columns=["Close time", "Quote volume", "Number of trades", "Taker base volume", "Taker quote volume"], inplace=True)
# btc_data.drop(columns=["Close time", "Quote volume", "Number of trades", "Taker base volume", "Taker quote volume"], inplace=True)
# eth_data.drop(columns=["Close time", "Quote volume", "Number of trades", "Taker base volume", "Taker quote volume"], inplace=True)

In [8]:
# eth_data.tail


In [9]:

df = eth_data
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,Quote volume,Trade count,Taker base volume,Taker quote volume
Open time,,,,,,,,,
2020-01-01 00:00:00+00:00,129.16,129.19,128.68,128.87,7769.17336,1.000930e+06,2504,4149.93345,5.346193e+05
2020-01-01 01:00:00+00:00,128.87,130.65,128.78,130.64,11344.65516,1.474278e+06,4885,5930.54276,7.704861e+05
2020-01-01 02:00:00+00:00,130.63,130.98,130.35,130.85,7603.35623,9.940256e+05,3046,3324.35218,4.346754e+05
2020-01-01 03:00:00+00:00,130.85,130.89,129.94,130.20,4968.55433,6.473610e+05,2818,1810.03564,2.358903e+05
2020-01-01 04:00:00+00:00,130.21,130.74,130.15,130.20,3397.90747,4.430067e+05,2264,1839.74371,2.398483e+05
...,...,...,...,...,...,...,...,...,...
2023-05-29 13:00:00+00:00,1898.20,1902.84,1895.80,1900.10,8232.89940,1.563808e+07,17187,3478.40000,6.607148e+06
2023-05-29 14:00:00+00:00,1900.09,1902.57,1885.00,1894.75,14439.17890,2.735379e+07,21559,6043.98480,1.144905e+07
2023-05-29 15:00:00+00:00,1894.74,1899.14,1886.31,1889.41,14199.72270,2.686451e+07,23161,7200.48690,1.362243e+07


## Generate a bunch of random entry signals

In [10]:
import numpy as np
df['signal'] = np.random.random(len(df))
df['signal'] = df['signal'].apply(lambda x: 1 if x<0.5 else 2) # 1 for buy, 2 for sell

In [11]:
df[df['signal']==1].count()+df[df['signal']==2].count()
#len(df)

Open                  29842
High                  29842
Low                   29842
Close                 29842
Volume                29842
Quote volume          29842
Trade count           29842
Taker base volume     29842
Taker quote volume    29842
signal                29842
dtype: int64

In [12]:
df.rename(columns={'Open time':'Datetime'}, inplace=True)

In [13]:
import numpy as np
test_df = df # df.loc['2021'] if you want to test on a specific time period

def SIGNAL():
    return test_df.signal

# df.columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'signal']

class TradingDown(Strategy):
    # Set up some variables to be used by the strategy these can all be optimized
    initsize = .1 # Shown in decimal ie. 0.10 = 10% how much to buy on the first trade
    addsize = .1 # Shown in decimal ie. 0.10 = 10% how much to add to the position
    multiplier = .35 # For position sizing addsize times multiplier if the last trade was a loss
    max_number_of_trades = 400 # Maximum number of trades to have open at any one time

    take_profit = 0.10 # Shown in decimal ie. 0.10 = 10%
    position_loss_threshold = 0.02 # how much does the position have to be down before we add to it Shown in decimal ie. 0.10 = 10% 
    trade_loss_threshold = 0.04 # how much does the trade have to be down before we add to it Shown in decimal ie. 0.10 = 10%
    


    def position_sizer(self) -> float:
        '''
        This is the position sizing function. It will be called on every trade to determine how much to buy or sell
        Currently the logic is to buy XX% of the portfolio on the first trade, then add to the trade if the position is down
        if the last trade is down more than the trade_loss_threshold then add to the position by addsize*multiplier
        '''
        size = 0.01  # Initialize as just 1percent of the portfolio
        # If we have no position, then buy the initsize
        if self.position.size == 0:
            size = self.initsize
        # If we are not down on the position by more than the position_loss_threshold, then return a tiny amount
        # elif self.position.pl_pct > -self.position_loss_threshold:
        #     size = 0.01  # Return tiny amount just 1% of the portfolio
        # If we are down on the position by more than the position_loss_threshold, then add to the position
        elif (self.position.pl_pct < -self.position_loss_threshold and
            len(self.trades) < self.max_number_of_trades):
            if self.trades[-1].pl_pct < -self.trade_loss_threshold: # Check the last trade to see if we are up or down on it
                size = self.addsize*self.multiplier
                return size
            else: 
                # If you are down on the position but the last trade is profitable, then add to the position by a fraction of the addsize
                # Comment the first one out and replace it with the second one to be more conservative
                size = self.addsize * 0.5 # Add a reduced 50% of the addsize 
                # size=0.01

        return size


    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        # Create some variables to make the code easier to read and for logging
        current_cash = self._broker._cash
        current_margin_available = self._broker.margin_available
        current_position_value = self.position.size*self.data.Close[-1]
        logging.info(f"Current cash: {current_cash} Current margin available: {current_margin_available} \
                     Current position value: {current_position_value},  \
                     Current Position PnL: {self.position.pl_pct}")
        
        # Long signals
        # Opening first trade in a group
        if self.signal1==1 and len(self.trades)==0:   # if the signal is 2 and we have no trades open
            tp1 = self.data.Close[-1] + self.data.Close[-1]*self.take_profit # Set the take profit price
            self.buy(tp=tp1, size=self.position_sizer()) # add sl=sl1,
            # print(f'Just bought {self.orders[-1].size} total cost: {self.position_sizer()*self.data.Close[-1]}')

        # Adding to a group of trades if the position is down
        elif (
            # if we have a position
            self.position and 
            # if the number of trades is less than the max number of trades
            len(self.trades) < self.max_number_of_trades and 
            # if the entire position's profit/loss percentage is less than the loss threshold
            self.position.pl_pct < -self.position_loss_threshold and 
            # if the profit/loss percentage of the last trade is less than the loss threshold
            self.trades[-1].pl_pct < -self.trade_loss_threshold
            ): 
            
            # sl1 = self.data.Close[-1] - self.data.Close[-1]*self.stop_loss_take_profit
            tp1 = self.data.Close[-1] + self.data.Close[-1]*self.take_profit # Set the take profit price
            self.buy(size=self.position_sizer()) # add sl=sl1, tp=tp1
           
        # If the entire group of trades is profitable then close the entire position
        elif self.position.pl_pct > self.take_profit:
            self.position.close()
            logging.info(f'Closed position {self.position.size} with pnl of {self.position.pl} at bar {self.data.index[-1]}')
        

        # Short signals
        # elif self.signal1==2 and len(self.trades) < self.max_number_of_trades:         
        #     # sl1 = self.data.Close[-1] + self.data.Close[-1]*self.stop_loss_take_profit
        #     tp1 = self.data.Close[-1] - self.data.Close[-1]*self.stop_loss_take_profit
        #     self.sell(tp=tp1, size=self.mysize) # add sl=sl1,

In [14]:


bt = Backtest(test_df, 
              TradingDown,
              cash=100000, 
              trade_on_close=True, 
              commission=.00, 
              exclusive_orders=False, 
              margin=1, # Set this to 0.5 for 2x leverage, 0.25 for 4x leverage, 0.125 for 8x leverage, etc.
              )
stat = bt.run()
print(stat)
bt.plot(resample=False)


Start                     2020-01-01 00:00...
End                       2023-05-29 17:00...
Duration                   1244 days 17:00:00
Exposure Time [%]                   99.805643
Equity Final [$]                   212266.517
Equity Peak [$]                    214910.037
Return [%]                         112.266517
Buy & Hold Return [%]             1364.537906
Return (Ann.) [%]                   24.674485
Volatility (Ann.) [%]               29.088332
Sharpe Ratio                         0.848261
Sortino Ratio                        1.563333
Calmar Ratio                         0.781392
Max. Drawdown [%]                  -31.577589
Avg. Drawdown [%]                   -0.568378
Max. Drawdown Duration      347 days 12:00:00
Avg. Drawdown Duration        2 days 23:00:00
# Trades                                  220
Win Rate [%]                             80.0
Best Trade [%]                      96.565394
Worst Trade [%]                    -46.433239
Avg. Trade [%]                    

/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1564', ...)

You can also re-run the backtest manually by changing parameters.
Here is the list of available parameters.
```python
    initsize = .1 # Shown in decimal ie. 0.10 = 10% how much to buy on the first trade
    addsize = .1 # Shown in decimal ie. 0.10 = 10% how much to add to the position
    multiplier = .35 # For position sizing addsize times multiplier if the last trade was a loss
    max_number_of_trades = 400 # Maximum number of trades to have open at any one time

    take_profit = 0.10 # Shown in decimal ie. 0.10 = 10%
    position_loss_threshold = 0.02 # how much does the position have to be down before we add to it Shown in decimal ie. 0.10 = 10% 
    trade_loss_threshold = 0.04 # how much does the trade have to be down before we add to it Shown in decimal ie. 0.10 = 10%
```

In [15]:
# Change a parameter and run the backtest again
new_run = bt.run(
    initsize = .05, # Shown in decimal ie. 0.10 = 10% how much to buy on the first trade
    addsize = .2, # Shown in decimal ie. 0.10 = 10% how much to add to the position
    multiplier = 1.5, # For position sizing addsize times multiplier if the last trade was a loss
    max_number_of_trades = 40, # Maximum number of trades to have open at any one time

    take_profit = 0.10, # Shown in decimal ie. 0.10 = 10%
    position_loss_threshold = 0.02, # how much does the position have to be down before we add to it Shown in decimal ie. 0.10 = 10% 
    trade_loss_threshold = 0.04, # how much does the trade have to be down before we add to it Shown in decimal ie. 0.10 = 10%
    )
print(new_run)
# bt.plot(resample=False)

Start                     2020-01-01 00:00...
End                       2023-05-29 17:00...
Duration                   1244 days 17:00:00
Exposure Time [%]                   99.798941
Equity Final [$]                   386442.721
Equity Peak [$]                    808136.841
Return [%]                         286.442721
Buy & Hold Return [%]             1364.537906
Return (Ann.) [%]                   48.624276
Volatility (Ann.) [%]              134.626971
Sharpe Ratio                         0.361178
Sortino Ratio                        0.887604
Calmar Ratio                         0.632523
Max. Drawdown [%]                  -76.873471
Avg. Drawdown [%]                   -1.142519
Max. Drawdown Duration      536 days 18:00:00
Avg. Drawdown Duration        2 days 09:00:00
# Trades                                  184
Win Rate [%]                        85.326087
Best Trade [%]                     154.740219
Worst Trade [%]                    -59.200451
Avg. Trade [%]                    

## Analyze the trades and cashflows

In [16]:
# print the trades
stat['_trades'].loc[stat['_trades']['PnL'] < 0] # Print only the trades that lost money

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
36,120,422,671,177.24,175.64,-192.00,-0.009027,2020-01-18 14:00:00+00:00,2020-01-28 23:00:00+00:00,10 days 09:00:00
78,80,1079,1167,285.96,280.60,-428.80,-0.018744,2020-02-15 00:00:00+00:00,2020-02-18 16:00:00+00:00,3 days 16:00:00
84,17,1690,2598,184.27,183.53,-12.58,-0.004016,2020-03-11 17:00:00+00:00,2020-04-18 13:00:00+00:00,37 days 20:00:00
85,17,1640,2598,194.50,183.53,-186.49,-0.056401,2020-03-09 15:00:00+00:00,2020-04-18 13:00:00+00:00,39 days 22:00:00
86,17,1623,2598,203.78,183.53,-344.25,-0.099372,2020-03-08 22:00:00+00:00,2020-04-18 13:00:00+00:00,40 days 15:00:00
...,...,...,...,...,...,...,...,...,...,...
782,6,27399,27991,1646.09,1643.34,-16.50,-0.001671,2023-02-16 22:00:00+00:00,2023-03-13 14:00:00+00:00,24 days 16:00:00
783,29,27394,27991,1715.69,1643.34,-2098.15,-0.042170,2023-02-16 17:00:00+00:00,2023-03-13 14:00:00+00:00,24 days 21:00:00
814,5,28925,29840,1907.53,1882.43,-125.50,-0.013158,2023-04-21 13:00:00+00:00,2023-05-29 16:00:00+00:00,38 days 03:00:00
815,5,28872,29840,1987.49,1882.43,-525.30,-0.052861,2023-04-19 08:00:00+00:00,2023-05-29 16:00:00+00:00,40 days 08:00:00


In [17]:
stat['_trades'][['PnL', 'ReturnPct', 'Duration']].describe() # Print some stats on the trades


,PnL,ReturnPct,Duration
count,817.000000,817.000000,817
mean,202.390431,0.025436,10 days 21:03:31.505507955
std,1496.438646,0.116277,27 days 14:23:32.909252842
min,-25562.550000,-0.483155,0 days 01:00:00
25%,229.418700,0.010000,0 days 02:00:00
50%,340.600000,0.010000,0 days 08:00:00
75%,474.169600,0.010000,3 days 12:00:00
max,3744.810000,0.942689,159 days 13:00:00


# Now let's run the optimization
Note the `skopt` method isn't working. I think skopt needs to be updated. I'm running this in grid mode so it'll take a while.

In [21]:
import numpy as np

optimize_result = bt.optimize(
    initsize                =np.arange(0.1, 0.3, 0.1).tolist(),
    addsize                 =np.arange(0.1,  0.3, 0.1).tolist(),
    take_profit             =np.arange(0.01, 0.05, 0.01).tolist(),
    position_loss_threshold =np.arange(0.01, 0.05, 0.01).tolist(),
    trade_loss_threshold    =np.arange(.01, .05, 0.01).tolist(),
    max_number_of_trades    =np.arange(10, 50, 5).tolist(),
    multiplier              =np.arange(0.5, 2, 0.5).tolist(),
    maximize='Equity Final [$]',
    method='grid',
    # method='skopt',
    max_tries=100,
    random_state=42,
    return_heatmap=True,


)


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/14 [00:00<?, ?it/s]

The above optimization took about 3 mins so pretty good. 

In [22]:
optimize_result

(Start                     2020-01-01 00:00...
 End                       2023-05-29 17:00...
 Duration                   1244 days 17:00:00
 Exposure Time [%]                   99.604584
 Equity Final [$]                  524407.9796
 Equity Peak [$]                  1176794.8596
 Return [%]                          424.40798
 Buy & Hold Return [%]             1364.537906
 Return (Ann.) [%]                   62.508616
 Volatility (Ann.) [%]               157.39653
 Sharpe Ratio                         0.397141
 Sortino Ratio                        1.082012
 Calmar Ratio                         0.796669
 Max. Drawdown [%]                  -78.462506
 Avg. Drawdown [%]                   -1.559241
 Max. Drawdown Duration      536 days 18:00:00
 Avg. Drawdown Duration        2 days 11:00:00
 # Trades                                  350
 Win Rate [%]                             86.0
 Best Trade [%]                     156.913682
 Worst Trade [%]                    -59.200451
 Avg. Trade [

If you are getting the np.int error read here. https://github.com/scikit-optimize/scikit-optimize/issues/1138 simple fix is to

``` python
import numpy as np
np.int = int
``` 

however that will only leave you with the next error which I show above. 
`InvalidParameterError: The 'criterion' parameter of ExtraTreesRegressor must be a str among {'squared_error', 'friedman_mse', 'absolute_error', 'poisson'}. Got 'mse' instead.`